<a href="https://colab.research.google.com/github/Shefalikaushal13/AmEx-User-Exit-Prediction/blob/main/AmEx_exit_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
tf.__version__

'2.18.0'

Dataset Import

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Amex User Exit.xlsx.csv to Amex User Exit.xlsx.csv


In [ ]:
import io

df = pd.read_csv(io.BytesIO(uploaded['Amex User Exit.xlsx.csv']))
print(df.head())


   Credit Score  Geography  Gender   Age  Customer Since  Current Account  \
0         553.0      Delhi  Female  45.0             4.0            0.000   
1         447.0  Bengaluru    Male  31.0             7.0            0.000   
2         501.0      Delhi  Female  32.0             2.0            0.000   
3         428.0      Delhi    Male  51.0             3.0            0.000   
4         492.0      Delhi  Female  57.0             6.0      1912681.501   

   Num of products  UPI Enabled  Estimated Yearly Income  Closed  
0              4.0          1.0                 274150.0     0.0  
1              4.0          1.0                 519360.0     0.0  
2              4.0          1.0                 545501.0     0.0  
3              4.0          1.0                  86868.0     0.0  
4              2.0          1.0                 518680.0     0.0  


In [ ]:
df['Closed'] = df['Closed'].fillna(df['Closed'].mean())  #to handle NaN values in variable y

In [ ]:
X=df.iloc[:,0:-1].values
y=df.iloc[:,-1].values

In [ ]:
print(df.isnull().sum())

Credit Score               2
Geography                  2
Gender                     2
Age                        2
Customer Since             2
Current Account            2
Num of products            2
UPI Enabled                2
Estimated Yearly Income    2
Closed                     0
dtype: int64


In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

In [ ]:
print(X)

[[0.00000000e+00 1.00000000e+00 0.00000000e+00 ... 4.00000000e+00
  1.00000000e+00 2.74150000e+05]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 4.00000000e+00
  1.00000000e+00 5.19360000e+05]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 ... 4.00000000e+00
  1.00000000e+00 5.45501000e+05]
 ...
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 ... 4.00000000e+00
  1.00000000e+00 1.80031000e+05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 3.02931399e+00
  7.04946107e-01 2.74356782e+05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 3.02931399e+00
  7.04946107e-01 2.74356782e+05]]


In [ ]:
print(y)

[0.         0.         0.         ... 0.         0.20429133 0.20429133]


Encoding Categorial Data


Label Encode Gender and One-Hot encode geography

In [ ]:
from sklearn.preprocessing import LabelEncoder  #doing for gender column
label_encoder=LabelEncoder()
X[:,2]=label_encoder.fit_transform(X[:,2])  #fit: The fit method of the LabelEncoder analyzes the unique values (categories) in the selected column (X[:, 2]) and assigns a unique integer label to each category.
#transform: The transform method then replaces the original categorical values in the column with their corresponding numerical labels.

In [ ]:
print(X)

[[0.00000000e+00 1.00000000e+00 0.00000000e+00 ... 4.00000000e+00
  1.00000000e+00 2.74150000e+05]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 4.00000000e+00
  1.00000000e+00 5.19360000e+05]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 ... 4.00000000e+00
  1.00000000e+00 5.45501000e+05]
 ...
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 ... 4.00000000e+00
  1.00000000e+00 1.80031000e+05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 3.02931399e+00
  7.04946107e-01 2.74356782e+05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 3.02931399e+00
  7.04946107e-01 2.74356782e+05]]


In [ ]:
from sklearn.compose import ColumnTransformer  #One-Hot encoding for geography
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X=np.array(ct.fit_transform(X))

In [ ]:
print(X[2])

[0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 5.01000e+02
 0.00000e+00 3.20000e+01 2.00000e+00 0.00000e+00 4.00000e+00 1.00000e+00
 5.45501e+05]


Splitting into training and testing set

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)
#fit: The fit method calculates the mean and standard deviation of each feature (column) in your training data (X_train).
#transform: The transform method then applies standardization to X_train by subtracting the mean and dividing by the standard deviation for each feature. This transforms the data so that it has a mean of 0 and a standard deviation of 1. The result is assigned back to X_train, overwriting the original data.

ANN Initialisation

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [ ]:
input_dim=X_train.shape[1]

In [ ]:
ann = Sequential([
    Dense(128, activation='relu', input_shape=(input_dim,)),  #This creates a densely connected layer (also called a fully connected layer) with 128 neurons.
    Dropout(0.3), #This adds a dropout layer with a rate of 0.3. Dropout randomly sets a fraction (0.3 in this case) of the input units to 0 during training. This helps prevent overfitting by forcing the network to learn more robust features.
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid') # This is the output layer of the model. It has a single neuron and uses the 'sigmoid' activation function. The sigmoid function outputs a value between 0 and 1, which is suitable for binary classification problems.
])

Compiling ANN

In [ ]:
from tensorflow.keras.optimizers import Adam
ann.compile(optimizer=Adam(learning_rate=0.0001),
            loss='binary_crossentropy',
            metrics=['accuracy'])
# Adam optimiser is a type of optimiser from keras
# Binary cross entropy mans that since we are trying to predict a binary value, cross entropy essentially means
# the entropy exachange between different neurons which we need to be binary.
# Metrics means the parameters which we want to imporve which is accuracy here.

In [ ]:
ann.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_11 (Dense)                     │ (None, 128)                 │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,161 (47.50 KB)

 Trainable params: 12,161 (47.50 KB)

 Non-trainable params: 0 (0.00 B)

Training on training set

In [ ]:
ann.fit(X_train,y_train,batch_size=32,epochs=120)
#we have a datset of 10k values, so we r telling the algo to not train them all at once
#Make batches of 32 and iterate each batch 120 times and update the weights accordingly

Epoch 1/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7174 - loss: 0.6003
Epoch 2/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7941 - loss: 0.4966
Epoch 3/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7931 - loss: 0.4787
Epoch 4/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7982 - loss: 0.4571
Epoch 5/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7986 - loss: 0.4468
Epoch 6/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8070 - loss: 0.4392
Epoch 7/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7974 - loss: 0.4445
Epoch 8/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8068 - loss: 0.4312
Epoch 9/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8104 - loss: 0.4272
Epoch 10/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8068 - loss: 0.4312
Epoch 11/120
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8185 - loss: 0.4205
Epoch 12/120
249/249 ━━━━━━━━━━━━━━━━━━━━

Predictions

In [ ]:
print(ann.predict(sc.transform([[0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 5.01000e+02,
 0.00000e+00, 3.20000e+01, 2.00000e+00, 0.00000e+00, 4.00000e+00, 1.00000e+00,
 5.45501e+05]]))>0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
[[False]]


Prediction on test set

In [ ]:
y_pred=ann.predict(X_test)
y_pred=(y_pred>0.5)

print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[0. 0.]
 [0. 0.]
 [1. 1.]
 ...
 [0. 0.]
 [0. 1.]
 [0. 1.]]


Confusion Matrix

In [ ]:
y_test_binary = (y_test > 0.5).astype(int)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test_binary,y_pred)
print(cm)
accuracy_score=accuracy_score(y_test_binary,y_pred)
print(accuracy_score)

[[1506   84]
 [ 224  172]]
0.8449144008056395


# New Section